## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import PIL

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, BatchNormalization, Layer, LeakyReLU

import warnings
warnings.filterwarnings("ignore")

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
# if len(physical_devices) > 0:
#   tf.config.experimental.set_memory_growth(physical_devices[0], True)

[]


In [2]:
def imshow(img: np.array):
  display(PIL.Image.fromarray(img))

## Loading dataset

In [3]:
ds, ds_info = tfds.load(
    "celeb_a",
    split='train[:10%]',
    shuffle_files=True,
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

DownloadError: ignored

In [ ]:
## ds is tensorflow object and each sample is dict with some features not only image, convert to numpy for simplicity 
ds = np.vstack(tfds.as_numpy(ds))
ds = np.array(list(map(lambda x: x[0]['image'], ds)))
ds.shape

In [ ]:
## padd images in order them be multiple divisible by 2 
right_pad = 224 - ds[0].shape[1]
bottom_pad = 224 - ds[0].shape[0]
ds = np.array(list(map(lambda x: np.pad(x, [(0, bottom_pad), (0, right_pad), (0, 0)], mode='constant', constant_values=255), ds)))
ds.shape

In [ ]:
idxs = np.random.choice(len(ds), 5)
imshow(np.concatenate(ds[idxs], 1))

## Masking

In [ ]:
def create_line_mask(img):
  mask = np.full(img.shape, 255, np.uint8)
  for _ in range(np.random.randint(4, 10)):
    x1, x2 = np.random.randint(1, img.shape[1]-right_pad), np.random.randint(1, img.shape[1]-right_pad)
    y1, y2 = np.random.randint(1, img.shape[0]-bottom_pad), np.random.randint(1, img.shape[0]-bottom_pad)
    thickness = np.random.randint(3, 8)
    cv2.line(mask, (x1, y1), (x2, y2), (1, 1, 1), thickness)

  masked_image = cv2.bitwise_and(img, mask)

  return masked_image

In [ ]:
idxs = np.random.choice(len(ds), 5)
masked = np.array(list(map(create_line_mask, ds[idxs])))
imshow(np.concatenate(masked, 1))

In [ ]:
def create_square_mask(img, size=80):
  mask = np.full(img.shape, 255, np.uint8)
  x1, y1 = np.random.randint(20, img.shape[1]-size-20-right_pad), np.random.randint(20, img.shape[0]-size-20-bottom_pad)
  cv2.rectangle(mask, (x1, y1), (x1+size, y1+size), (1, 1, 1), -1)

  masked_image = cv2.bitwise_and(img, mask)

  return masked_image

In [ ]:
idxs = np.random.choice(len(ds), 5)
masked = np.array(list(map(create_square_mask, ds[idxs])))
imshow(np.concatenate(masked, 1))

## Splitting dataset

In [ ]:
def train_val_test_split(ds, train_size=0.8, val_size=0.9):   
  train_split = int(train_size * len(ds))
  val_split = int(val_size * len(ds))

  train = ds[:train_split]
  val = ds[train_split:val_split]
  test = ds[val_split:]

  return train, val, test

## Autoencoder

In [ ]:
class Autoencoder(keras.Model):
  def __init__(self):
    super(Autoencoder, self).__init__()


  def __ConvBlock(self, out, kernel_size, prev_layer):
    cnn = Conv2D(out, kernel_size, activation="relu", padding="same")(prev_layer)
    cnn = BatchNormalization()(cnn)
    cnn = LeakyReLU()(cnn)
    return cnn

  def __EncodeBlock(self, out, kernel_size, prev_layer):
    conv = self.__ConvBlock(out, kernel_size, prev_layer)
    conv = self.__ConvBlock(out, kernel_size, conv)
    conv = self.__ConvBlock(out, kernel_size, conv)
    conv = MaxPooling2D((2, 2))(conv)
    return conv

  
  def __DecodeBlock(self, out, kernel_size, prev_layer):
    up = Conv2DTranspose(out, kernel_size, strides=(2, 2), padding="same")(prev_layer)
    up = BatchNormalization()(up)
    up = LeakyReLU()(up)
    return up

  
  def model(self, input_shape=(224, 224, 3)):
    inputs = keras.layers.Input(input_shape)

    conv1 = self.__EncodeBlock(32, (3,3), inputs) 
    conv2 = self.__EncodeBlock(64, (3,3), conv1)
    conv3 = self.__EncodeBlock(128, (3,3), conv2) 
    conv4 = self.__EncodeBlock(256, (3,3), conv3) 
    
    deconv1 = self.__DecodeBlock(256, (3,3), conv4)
    deconv2 = self.__DecodeBlock(128, (3,3), deconv1)
    deconv3 = self.__DecodeBlock(64, (3,3), deconv2)
    deconv4 = self.__DecodeBlock(32, (3,3), deconv3)
    
    outputs = keras.layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(deconv4)

    return keras.models.Model(inputs=[inputs], outputs=[outputs])

In [ ]:
model = Autoencoder().model()

In [ ]:
# keras.utils.plot_model(model, show_shapes=True, dpi=76)

In [ ]:
model.compile(
  optimizer=keras.optimizers.Adam(),
  loss=keras.losses.MeanAbsoluteError(),
)

In [ ]:
train, val, test = train_val_test_split(ds)
train.shape, val.shape, test.shape

In [ ]:
model.fit(test, test, batch_size=64, epochs=2)